## Two option to download data from ICESat-2

- It is working on 11.11.2025

# Option 1:

- download all tracks in a AOI

In [27]:
# --- ICESat-2 download via earthaccess with 2D/3D-safe AOI polygon ---
# pip install earthaccess geopandas shapely pyproj fiona

import os, json
import geopandas as gpd
from shapely.ops import unary_union
from shapely.geometry import Polygon, MultiPolygon, LineString, Point, mapping
from shapely.validation import make_valid
import earthaccess as ea

# ------------------ USER SETTINGS ------------------
SHAPEFILE_PATH = r"C:\coding\arctic\paper1\shp\AOI_Drew.shp"
OUTPUT_DIR     = r"C:\coding\arctic\data"
SHORT_NAME     = "ATL03"      # or ATL03/ATL08/ATL12...
VERSION        = "007"        # use "007" for current ATL06; or None
TIME_START     = "2020-01-01"
TIME_END       = "2020-11-10"
CLOUD_HOSTED   = True         # set False to use NSIDC Direct instead of AWS cloud host
# ---------------------------------------------------

def to_single_polygon(g):
    """Coerce any geometry into a single valid Polygon for CMR."""
    if g.is_empty:
        raise ValueError("AOI geometry is empty.")
    if isinstance(g, Polygon):
        poly = g
    elif isinstance(g, MultiPolygon):
        poly = max(g.geoms, key=lambda x: x.area)
    elif isinstance(g, LineString):
        poly = g.buffer(0.001)  # ~100 m at equator; AOI just needs a footprint
    elif isinstance(g, Point):
        poly = g.buffer(0.001)
    else:
        poly = g.convex_hull
    # fix self-intersections etc.
    poly = make_valid(poly)
    if isinstance(poly, MultiPolygon):
        poly = max(poly.geoms, key=lambda x: x.area)
    if not isinstance(poly, Polygon) or poly.is_empty:
        raise ValueError("Could not produce a valid polygon from AOI.")
    return poly

from shapely.geometry import Polygon
from shapely.geometry.polygon import orient

def cmr_polygon_coords(poly, max_points=500):
    """
    Return exterior ring as list of [lon, lat] PAIRS for CMR, CLOSED (last==first),
    explicitly oriented counter-clockwise. Handles 2D/3D coords; simplifies if needed.
    """
    if not isinstance(poly, Polygon):
        raise ValueError("AOI is not a Polygon.")

    # 1) Force CCW orientation
    poly_ccw = orient(poly, sign=1.0)

    # 2) Simplify if vertex count too high, then enforce CCW again (simplify can flip it)
    candidate = poly_ccw.simplify(0.0002, preserve_topology=True) \
                if len(poly_ccw.exterior.coords) > max_points else poly_ccw
    candidate = orient(candidate, sign=1.0)

    ring = candidate.exterior

    # 3) Build lon/lat pairs (support 2D/3D), normalize longitudes
    coords = []
    for c in ring.coords:
        x = float(c[0]); y = float(c[1])
        if x > 180: x -= 360
        if x < -180: x += 360
        coords.append((x, y))

    # 4) Close the ring (CMR requires last==first) and basic checks
    if len(coords) < 4:
        raise ValueError(f"CMR polygon needs >= 4 coordinate pairs; got {len(coords)}.")
    if coords[0] != coords[-1]:
        coords.append(coords[0])
    if len(coords) < 5:
        raise ValueError("Closed polygon must contain at least 4 unique vertices (5 with closure).")

    return coords


# 1) Read AOI and ensure WGS84
gdf = gpd.read_file(SHAPEFILE_PATH)
if gdf.crs is None:
    raise ValueError("Shapefile has no CRS. Define/reproject it, then retry.")
if gdf.crs.to_epsg() != 4326:
    gdf = gdf.to_crs(epsg=4326)

# 2) Merge into a single polygon
geom = unary_union(gdf.geometry)
poly = to_single_polygon(geom)
coords = cmr_polygon_coords(poly)
print(f"AOI exterior vertices used: {len(coords)}")

# 3) Auth & search
ea.login(strategy="netrc")  # prompts if no ~/.netrc
kwargs = {
    "short_name": SHORT_NAME,
    "temporal": (TIME_START, TIME_END),
    "polygon": coords,              # <-- CMR expects a flat list of lon/lat pairs
    "cloud_hosted": CLOUD_HOSTED
}
if VERSION is not None:
    kwargs["version"] = VERSION

results = ea.search_data(**kwargs)
print(f"Found {len(results)} {SHORT_NAME} granule(s) with polygon filter.")


AOI exterior vertices used: 5
Found 38 ATL03 granule(s) with polygon filter.


In [ ]:
# 4) Download (fallback to bbox if polygon returned none)
os.makedirs(OUTPUT_DIR, exist_ok=True)
if results:
    downloads = ea.download(results, local_path=OUTPUT_DIR)
    print(f"Downloaded {len(downloads)} file(s) to: {OUTPUT_DIR}")
else:
    minx, miny, maxx, maxy = poly.bounds
    print("No polygon results; retrying with bounding box…")
    bb_args = {
        "short_name": SHORT_NAME,
        "temporal": (TIME_START, TIME_END),
        "bounding_box": (minx, miny, maxx, maxy),
        "cloud_hosted": CLOUD_HOSTED
    }
    if VERSION is not None:
        bb_args["version"] = VERSION
    results_bb = ea.search_data(**bb_args)
    print(f"Found {len(results_bb)} with bounding box.")
    if results_bb:
        downloads = ea.download(results_bb, local_path=OUTPUT_DIR, flatten=True)
        print(f"Downloaded {len(downloads)} file(s) to: {OUTPUT_DIR}")

# 5) Save query metadata
meta = {
    "short_name": SHORT_NAME,
    "version": VERSION,
    "temporal": {"start": TIME_START, "end": TIME_END},
    "polygon_coords_len": len(coords),
    "output_dir": OUTPUT_DIR,
    "cloud_hosted": CLOUD_HOSTED
}
with open(os.path.join(OUTPUT_DIR, f"{SHORT_NAME}_download_query.json"), "w") as f:
    json.dump(meta, f, indent=2)
print("Saved query metadata.")

# Option 2:

- Download a selected track(or tracks) for a AOI

In [8]:
# --- ICESat-2 download via earthaccess with AOI shapefile + multi-track filtering ---
# Install once (separate cell / terminal):
#   pip install earthaccess geopandas shapely pyproj fiona

import os, re, json
import geopandas as gpd
from shapely.ops import unary_union
from shapely.geometry import Polygon, MultiPolygon, LineString, Point
from shapely.validation import make_valid
from shapely.geometry.polygon import orient
import earthaccess as ea

# ===================== USER SETTINGS =====================
SHAPEFILE_PATH = r"C:\coding\arctic\paper1\shp\AOI_Drew_CLEAN.shp"
OUTPUT_DIR     = r"C:\coding\arctic\paper1\ATL06_all\ATL03\0129"

SHORT_NAME     = "ATL03"      # e.g., ATL03 (photons), ATL06 (heights), ATL08, ATL12
VERSION        = "007"        # set None to accept any (ATL06 usually "007")
TIME_START     = "2019-02-01"
TIME_END       = "2019-04-30"
CLOUD_HOSTED   = True         # True: prefer AWS cloud-hosted; False: NSIDC Direct

# —— Choose tracks ——
# Use None to download ALL tracks that intersect your AOI/time, or a list like ["129","731"]
TRACKS         = ["137"]    # <- example; ints or strings OK; they’ll be zero-padded to 4 digits
SPLIT_BY_TRACK = True              # save each track to its own subfolder under OUTPUT_DIR
# =========================================================

# ---------------- Helpers ----------------
def to_single_polygon(g):
    """Coerce any geometry into a single valid Polygon for CMR."""
    if g is None or g.is_empty:
        raise ValueError("AOI geometry is empty.")
    if isinstance(g, Polygon):
        poly = g
    elif isinstance(g, MultiPolygon):
        poly = max(g.geoms, key=lambda x: x.area)
    elif isinstance(g, LineString):
        poly = g.buffer(0.001)  # ~100 m at equator
    elif isinstance(g, Point):
        poly = g.buffer(0.001)
    else:
        poly = g.convex_hull
    poly = make_valid(poly)
    if isinstance(poly, MultiPolygon):
        poly = max(poly.geoms, key=lambda x: x.area)
    if not isinstance(poly, Polygon) or poly.is_empty:
        raise ValueError("Could not produce a valid polygon from AOI.")
    return poly

def cmr_polygon_coords(poly, max_points=500):
    """
    Return exterior ring as list of [lon, lat] pairs for CMR, CLOSED and CCW.
    Handles 2D/3D coords and simplifies if too many vertices.
    """
    if not isinstance(poly, Polygon):
        raise ValueError("AOI is not a Polygon.")
    # Force CCW orientation
    poly = orient(poly, sign=1.0)
    # Simplify if too many vertices; enforce CCW again (simplify may flip)
    if len(poly.exterior.coords) > max_points:
        poly = orient(poly.simplify(0.0002, preserve_topology=True), sign=1.0)

    coords = []
    for c in poly.exterior.coords:
        x = float(c[0]); y = float(c[1])
        # normalize longitudes to [-180, 180] just in case
        if x > 180: x -= 360
        if x < -180: x += 360
        coords.append((x, y))

    if len(coords) < 4:
        raise ValueError(f"CMR polygon needs >= 4 coordinate pairs; got {len(coords)}.")
    # close ring (CMR requires last==first)
    if coords[0] != coords[-1]:
        coords.append(coords[0])
    if len(coords) < 5:
        raise ValueError("Closed polygon must contain at least 4 unique vertices (5 with closure).")
    return coords

def granule_name(g):
    """Prefer ProducerGranuleId or GranuleUR; fallback to any id-like string."""
    umm = getattr(g, "umm", None) or (g.get("umm", {}) if isinstance(g, dict) else {})
    if isinstance(umm, dict):
        dg  = umm.get("DataGranule", {}) or {}
        pid = dg.get("ProducerGranuleId")
        if pid: return str(pid)
        gur = umm.get("GranuleUR")
        if gur: return str(gur)
    return str(getattr(g, "granule_id", "") or getattr(g, "id", "") or g)

def extract_track_from_name(name: str):
    """
    Get the 4-digit RGT from an ICESat-2 granule name.
    Prefer underscore-splitting (robust), then regex fallback.
    Examples:
      ATL03_20190105212430_01290203_007_01.h5  -> 0129
      ATL06_20230714T101530_07310115_007_02.h5 -> 0731
    """
    s = str(name)
    parts = s.split("_")
    # prefer 8-digit (RGT+cycle) block: take first 4 digits as RGT
    for p in parts:
        if p.isdigit() and len(p) == 8:
            return p[:4]
    # then plain 4-digit block
    for p in parts:
        if p.isdigit() and len(p) == 4:
            return p
    # regex fallback (odd separators)
    m = re.search(r"[_.-](\d{8})[_.-]", s)
    if m: return m.group(1)[:4]
    m = re.search(r"[_.-](\d{4})[_.-]", s)
    if m: return m.group(1)
    return None

def extract_track(g):
    """Extract 4-digit RGT from a granule object using the filename."""
    rgt = extract_track_from_name(granule_name(g))
    return rgt.zfill(4) if rgt else None

def z4(x): 
    return str(x).zfill(4)

# ---------------- Main ----------------
# 1) Read AOI and ensure WGS84
gdf = gpd.read_file(SHAPEFILE_PATH)
if gdf.crs is None:
    raise ValueError("Shapefile has no CRS. Define/reproject it, then retry.")
if gdf.crs.to_epsg() != 4326:
    gdf = gdf.to_crs(epsg=4326)

# 2) AOI -> polygon -> coords
geom = unary_union(gdf.geometry)
poly = to_single_polygon(geom)
coords = cmr_polygon_coords(poly)
print(f"AOI vertices: {len(coords)} | Bounds: {poly.bounds}")

# 3) Auth & search
ea.login(strategy="netrc")  # prompts if not configured

kwargs = {
    "short_name": SHORT_NAME,
    "temporal": (TIME_START, TIME_END),
    "polygon": coords,
    "cloud_hosted": CLOUD_HOSTED
}
if VERSION is not None:
    kwargs["version"] = VERSION

# 4) Search by polygon; fallback to bbox if needed
try:
    results = ea.search_data(**kwargs)
except Exception as e:
    print(f"Polygon search raised: {e}\nFalling back to bounding box…")
    minx, miny, maxx, maxy = poly.bounds
    kwargs_bb = {
        "short_name": SHORT_NAME,
        "temporal": (TIME_START, TIME_END),
        "bounding_box": (minx, miny, maxx, maxy),
        "cloud_hosted": CLOUD_HOSTED
    }
    if VERSION is not None:
        kwargs_bb["version"] = VERSION
    results = ea.search_data(**kwargs_bb)

print(f"Found {len(results)} {SHORT_NAME} granule(s) before track filtering.")
for g in results[:5]:
    print("  •", granule_name(g))

# 5) Multi-track filter & download
requested = None if (not TRACKS) else {z4(t) for t in TRACKS}
print("Requested tracks:", sorted(requested) if requested else "(all)")

# Parse RGT for each result
by_track = {}
unknown = []
for g in results:
    rgt = extract_track(g)  # returns 4-digit string or None
    if rgt is None:
        unknown.append(g)
    else:
        by_track.setdefault(rgt, []).append(g)

print(f"Parsed tracks found: {sorted(by_track.keys())}  |  unknown={len(unknown)}")

# Decide batches to download
if not requested:
    batches = {"ALL": results}  # no filtering
else:
    batches = {rgt: by_track.get(rgt, []) for rgt in requested}
    missing = [r for r in requested if not batches[r]]
    if missing:
        print("No granules for tracks:", sorted(missing))

# print how many tracks to download
print(f"Downloading {len(batches)} track(s):", sorted(batches.keys()))

# print number of files per track
for rgt, granules in batches.items():
    print(f"  Track {rgt}: {len(granules)} file(s)")


AOI vertices: 5 | Bounds: (-153.93582877221303, 70.872503521221, -153.673676720986, 70.89384248884699)
Found 5 ATL03 granule(s) before track filtering.
  • ATL03_20190203200043_05710203_007_01.h5
  • ATL03_20190308182822_10740203_007_01.h5
  • ATL03_20190309071315_10820205_007_01.h5
  • ATL03_20190406170422_01290303_007_01.h5
  • ATL03_20190407054915_01370305_007_01.h5
Requested tracks: ['0137']
Parsed tracks found: ['0129', '0137', '0571', '1074', '1082']  |  unknown=0
  Track 0137: 1 file(s)


In [9]:
# 6) Download
os.makedirs(OUTPUT_DIR, exist_ok=True)
total = 0
for rgt, granules in batches.items():
    if not granules:
        continue
    subdir = os.path.join(OUTPUT_DIR, f"{rgt}") if SPLIT_BY_TRACK and rgt != "ALL" else OUTPUT_DIR
    os.makedirs(subdir, exist_ok=True)
    try:
        dl = ea.download(granules, local_path=subdir, keep_structure=False)
    except TypeError:
        dl = ea.download(granules, local_path=subdir)
    print(f"Downloaded {len(dl)} file(s) for track {rgt} → {subdir}")
    total += len(dl)

if total == 0:
    print("No granules to download. Consider adjusting time window, version, AOI, or requested tracks.")

# 7) Save query metadata
meta = {
    "short_name": SHORT_NAME,
    "version": VERSION,
    "temporal": {"start": TIME_START, "end": TIME_END},
    "polygon_coords_len": len(coords),
    "output_dir": OUTPUT_DIR,
    "cloud_hosted": CLOUD_HOSTED,
    "tracks_requested": (sorted(requested) if requested else "(all)"),
    "tracks_found": sorted(by_track.keys())
}
with open(os.path.join(OUTPUT_DIR, f"{SHORT_NAME}_download_query.json"), "w") as f:
    json.dump(meta, f, indent=2)
print("Saved query metadata.")

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

Downloaded 1 file(s) for track 0137 → C:\coding\arctic\paper1\ATL06_all\ATL03\0129\0137
Saved query metadata.


## New version

In [4]:
import pathlib as pl

OUTPUT_DIR_RAW = r"C:\coding\arctic\paper1\ATL06_all\ATL03\0129\0129"
files = list(pl.Path(OUTPUT_DIR_RAW).glob("*.h5"))

print("Found", len(files), "raw ATL03 files")


Found 1 raw ATL03 files


In [1]:
# import h5py
# import geopandas as gpd
# from shapely.geometry import Point
# from shapely.vectorized import contains
# import numpy as np
# import os

# # INPUT: same AOI from your old cell
# # AOI = polygon from the old download cell

# CLIPPED_DIR = r"C:\coding\arctic\paper1\ATL06_all\ATL03_clipped"
# os.makedirs(CLIPPED_DIR, exist_ok=True)

# BEAMS = ["gt1l","gt1r","gt2l","gt2r","gt3l","gt3r"]

# minx, miny, maxx, maxy = AOI.bounds

# def clip_one(path):
#     print(f"\nClipping: {os.path.basename(path)}")

#     with h5py.File(path, "r") as f:
#         for beam in BEAMS:
#             grp = f.get(f"{beam}/heights")
#             if grp is None:
#                 continue

#             lat = grp["lat_ph"][:]
#             lon = grp["lon_ph"][:]
#             h   = grp["h_ph"][:]
#             conf = grp["signal_conf_ph"][:]

#             # Fast bounding box filter
#             m1 = (
#                 (lon >= minx) & (lon <= maxx) &
#                 (lat >= miny) & (lat <= maxy)
#             )
#             if m1.sum() == 0:
#                 continue

#             lon2, lat2 = lon[m1], lat[m1]
#             h2         = h[m1]
#             conf2      = conf[m1]

#             # Accurate polygon filter
#             m2 = contains(AOI, lon2, lat2)
#             if m2.sum() == 0:
#                 continue

#             gdf = gpd.GeoDataFrame(
#                 {
#                     "longitude": lon2[m2],
#                     "latitude":  lat2[m2],
#                     "h_ph":      h2[m2],
#                     "signal_conf": list(conf2[m2]),
#                     "beam": beam,
#                     "granule": os.path.basename(path)
#                 },
#                 geometry=[Point(x, y) for x, y in zip(lon2[m2], lat2[m2])],
#                 crs="EPSG:4326"
#             )

#             outfile = os.path.basename(path).replace(".h5", f"_{beam}.parquet")
#             gdf.to_parquet(os.path.join(CLIPPED_DIR, outfile))

#             print(f"   saved {outfile} ({len(gdf)} photons)")


# # Run clipping for all downloaded files
# for f in files:   # <-- 'files' comes from your OLD download cell
#     clip_one(f)
# # ============================================================
# # CLIP ATL03 PHOTONS TO AOI — Save as .parquet AND simplified .h5
# # ============================================================

# import pathlib as pl
# import h5py
# import geopandas as gpd
# from shapely.geometry import Point
# from shapely.vectorized import contains
# import numpy as np
# import os

# # ------------------------------------------------------------
# # 1) Load AOI polygon (this must already exist from old cell)
# # ------------------------------------------------------------
# # AOI = <polygon from download cell>
# minx, miny, maxx, maxy = AOI.bounds

# # ------------------------------------------------------------
# # 2) Load downloaded RAW ATL03 files from disk
# # ------------------------------------------------------------
# OUTPUT_DIR_RAW = pl.Path(r"C:\coding\arctic\paper1\ATL06_all\ATL03\0129\0129")
# files = list(OUTPUT_DIR_RAW.glob("*.h5"))
# print("Found", len(files), "raw ATL03 files")

# # ------------------------------------------------------------
# # 3) Output directories
# # ------------------------------------------------------------
# CLIPPED_PARQUET_DIR = pl.Path(r"C:\coding\arctic\paper1\ATL06_all\ATL03_clipped_parquet")
# CLIPPED_H5_DIR      = pl.Path(r"C:\coding\arctic\paper1\ATL06_all\ATL03_clipped_h5")

# CLIPPED_PARQUET_DIR.mkdir(exist_ok=True)
# CLIPPED_H5_DIR.mkdir(exist_ok=True)

# # ------------------------------------------------------------
# # 4) Beams
# # ------------------------------------------------------------
# BEAMS = ["gt1l","gt1r","gt2l","gt2r","gt3l","gt3r"]

# # ------------------------------------------------------------
# # 5) Function to write simplified clipped beam to HDF5
# # ------------------------------------------------------------
# def save_to_h5(gdf, granule_name, beam):
#     """
#     Save clipped photons to a simplified HDF5 file with structure:
#        /beam
#            lat_ph
#            lon_ph
#            h_ph
#            signal_conf_ph
#     """
#     h5_path = CLIPPED_H5_DIR / f"{granule_name}_{beam}.h5"

#     with h5py.File(h5_path, "w") as h5:
#         grp = h5.create_group(beam)
#         grp.create_dataset("lat_ph", data=gdf["latitude"].values)
#         grp.create_dataset("lon_ph", data=gdf["longitude"].values)
#         grp.create_dataset("h_ph", data=gdf["h_ph"].values)

#         # Convert list-of-arrays to 2D numpy array for HDF5
#         conf_arr = np.vstack(gdf["signal_conf"].values)
#         grp.create_dataset("signal_conf_ph", data=conf_arr)

#     print(f"   Saved HDF5: {h5_path.name}")


# # ------------------------------------------------------------
# # 6) Main clip function
# # ------------------------------------------------------------
# def clip_one(path):
#     print(f"\nClipping: {os.path.basename(path)}")
#     granule_name = os.path.basename(path).replace(".h5","")

#     with h5py.File(path, "r") as f:

#         for beam in BEAMS:
#             grp = f.get(f"{beam}/heights")
#             if grp is None:
#                 continue

#             # Load ATL03 photon data
#             lat  = grp["lat_ph"][:]
#             lon  = grp["lon_ph"][:]
#             h    = grp["h_ph"][:]
#             conf = grp["signal_conf_ph"][:]

#             # -----------------------------
#             # Step 1: Fast bounding-box mask
#             # -----------------------------
#             m1 = (
#                 (lon >= minx) & (lon <= maxx) &
#                 (lat >= miny) & (lat <= maxy)
#             )
#             if m1.sum() == 0:
#                 continue

#             lat2, lon2 = lat[m1], lon[m1]
#             h2         = h[m1]
#             conf2      = conf[m1]

#             # -----------------------------
#             # Step 2: Exact polygon mask
#             # -----------------------------
#             m2 = contains(AOI, lon2, lat2)
#             if m2.sum() == 0:
#                 continue

#             # -----------------------------
#             # Step 3: Build GeoDataFrame
#             # -----------------------------
#             gdf = gpd.GeoDataFrame(
#                 {
#                     "latitude": lat2[m2],
#                     "longitude": lon2[m2],
#                     "h_ph": h2[m2],
#                     "signal_conf": list(conf2[m2]),
#                     "beam": beam,
#                     "granule": granule_name
#                 },
#                 geometry=[Point(x, y) for x, y in zip(lon2[m2], lat2[m2])],
#                 crs="EPSG:4326"
#             )

#             # -----------------------------
#             # Step 4: Save .parquet
#             # -----------------------------
#             parquet_name = f"{granule_name}_{beam}.parquet"
#             outfile_parquet = CLIPPED_PARQUET_DIR / parquet_name
#             gdf.to_parquet(outfile_parquet)
#             print(f"   Saved Parquet: {parquet_name} ({len(gdf)} photons)")

#             # -----------------------------
#             # Step 5: Save simplified .h5
#             # -----------------------------
#             save_to_h5(gdf, granule_name, beam)


# # ------------------------------------------------------------
# # 7) Run clipping over all files
# # ------------------------------------------------------------
# for f in files:
#     clip_one(f)


In [11]:
# ============================================================
# ATL03 CLIPPING — ONE CLEAN HDF5 PER GRANULE WITH ALL 6 BEAMS
# ============================================================

import pathlib as pl
import h5py
import geopandas as gpd
from shapely.geometry import Point
from shapely.vectorized import contains
import numpy as np
import os
import pandas as pd

# ------------------------------------------------------------
# 1. Load AOI shapefile (this ensures AOI is correctly defined)
# ------------------------------------------------------------
AOI_SHP = r"C:\coding\arctic\paper1\shp\AOI_Drew_CLEAN.shp"
aoi_gdf = gpd.read_file(AOI_SHP)
if aoi_gdf.crs.to_epsg() != 4326:
    aoi_gdf = aoi_gdf.to_crs(4326)
AOI = aoi_gdf.unary_union

minx, miny, maxx, maxy = AOI.bounds
print("AOI loaded. Bounds:", AOI.bounds)

# ------------------------------------------------------------
# 2. Input raw ATL03 granules
# ------------------------------------------------------------
RAW_DIR = pl.Path(r"C:\coding\arctic\paper1\ATL06_all\ATL03\0129\0137")
files = list(RAW_DIR.glob("*.h5"))
print("Found", len(files), "raw ATL03 granules")

# ------------------------------------------------------------
# 3. Output directory for clipped results
# ------------------------------------------------------------
OUT_DIR = pl.Path(r"C:\coding\arctic\paper1\ATL06_all\ATL03")
OUT_DIR.mkdir(exist_ok=True)

# ------------------------------------------------------------
# 4. Beams
# ------------------------------------------------------------
BEAMS = ["gt1l","gt1r","gt2l","gt2r","gt3l","gt3r"]

# ------------------------------------------------------------
# 5. Clip a single granule and save ONE multi-beam HDF5
# ------------------------------------------------------------
def clip_granule(path):
    granule = path.stem
    print(f"\nProcessing: {granule}")

    # create one output h5 file for all beams
    out_h5 = OUT_DIR / f"{granule}_clipped.h5"
    h5_out = h5py.File(out_h5, "w")

    # optional: merged parquet with all beams
    merged_list = []

    with h5py.File(path, "r") as f:

        for beam in BEAMS:

            grp = f.get(f"{beam}/heights")
            if grp is None:
                continue

            lat  = grp["lat_ph"][:]
            lon  = grp["lon_ph"][:]
            h    = grp["h_ph"][:]
            conf = grp["signal_conf_ph"][:]

            # ---- Step 1: Fast bounding box filter ----
            bb_mask = (
                (lon >= minx) & (lon <= maxx) &
                (lat >= miny) & (lat <= maxy)
            )
            if bb_mask.sum() == 0:
                continue

            lat2 = lat[bb_mask]
            lon2 = lon[bb_mask]
            h2   = h[bb_mask]
            conf2 = conf[bb_mask]

            # ---- Step 2: Exact AOI polygon filter ----
            poly_mask = contains(AOI, lon2, lat2)

            if poly_mask.sum() == 0:
                continue

            # Final clipped data
            lat3  = lat2[poly_mask]
            lon3  = lon2[poly_mask]
            h3    = h2[poly_mask]
            conf3 = conf2[poly_mask]

            print(f"  Beam {beam}: {len(lat3)} photons")

            # ---- Save inside the multi-beam H5 ----
            bgrp = h5_out.create_group(beam)
            bgrp.create_dataset("lat_ph", data=lat3)
            bgrp.create_dataset("lon_ph", data=lon3)
            bgrp.create_dataset("h_ph", data=h3)
            bgrp.create_dataset("signal_conf_ph", data=conf3)

            # ---- Save for optional parquet merging ----
            merged_list.append(
                gpd.GeoDataFrame(
                    {
                        "latitude": lat3,
                        "longitude": lon3,
                        "h_ph": h3,
                        "beam": beam,
                        "granule": granule,
                        "signal_conf": list(conf3),
                    },
                    geometry=[Point(x, y) for x,y in zip(lon3, lat3)],
                    crs="EPSG:4326"
                )
            )

    h5_out.close()
    print("Saved:", out_h5.name)

    # -------- Optional merged parquet --------
    if len(merged_list) > 0:
        merged = gpd.GeoDataFrame( pd.concat(merged_list, ignore_index=True), crs="EPSG:4326")
        merged.to_parquet(OUT_DIR / f"{granule}_clipped.parquet")
        print("Saved merged parquet")

# ------------------------------------------------------------
# 6. Run clipping for all granules
# ------------------------------------------------------------
for f in files:
    clip_granule(f)


C:\Users\rbendopa\AppData\Local\Temp\ipykernel_13560\1094105240.py:21: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  AOI = aoi_gdf.unary_union


AOI loaded. Bounds: (-153.93582877221303, 70.872503521221, -153.673676720986, 70.89384248884699)
Found 2 raw ATL03 granules

Processing: ATL03_20190407054915_01370305_007_01
  Beam gt2l: 13721 photons
  Beam gt2r: 4339 photons
  Beam gt3l: 16666 photons
  Beam gt3r: 4541 photons
Saved: ATL03_20190407054915_01370305_007_01_clipped.h5
Saved merged parquet

Processing: ATL03_20241225020311_01372605_007_01
  Beam gt2l: 10380 photons
  Beam gt2r: 3069 photons
  Beam gt3l: 13016 photons
  Beam gt3r: 3240 photons
Saved: ATL03_20241225020311_01372605_007_01_clipped.h5
Saved merged parquet


In [17]:
import geopandas as gpd
import pathlib as pl
import pandas as pd

CLIPPED_DIR = pl.Path(r"C:\coding\arctic\paper1\ATL06_all\ATL03_clipped")

# Load all clipped beams for profile analysis
all_gdfs = []
for p in CLIPPED_DIR.glob("*.parquet"):
    gdf = gpd.read_parquet(p)
    all_gdfs.append(gdf)

atl03_clipped = gpd.GeoDataFrame(pd.concat(all_gdfs, ignore_index=True), crs="EPSG:4326")

atl03_clipped.head()


,longitude,latitude,h_ph,signal_conf,beam,granule,geometry
0,-153.895411,70.787374,2.876066,"[4, 4, 4, 4, 4]",gt1l,ATL03_20190105212430_01290203_007_01.h5,POINT (-153.89541 70.78737)
1,-153.895411,70.787374,2.884471,"[4, 4, 4, 4, 4]",gt1l,ATL03_20190105212430_01290203_007_01.h5,POINT (-153.89541 70.78737)
2,-153.895411,70.787374,3.036654,"[4, 4, 4, 4, 4]",gt1l,ATL03_20190105212430_01290203_007_01.h5,POINT (-153.89541 70.78737)
3,-153.895411,70.787374,2.988105,"[4, 4, 4, 4, 4]",gt1l,ATL03_20190105212430_01290203_007_01.h5,POINT (-153.89541 70.78737)
4,-153.895411,70.787374,2.942522,"[4, 4, 4, 4, 4]",gt1l,ATL03_20190105212430_01290203_007_01.h5,POINT (-153.89541 70.78737)
